<a href="https://colab.research.google.com/github/shinjeongdong/MLDeeplearningStudy/blob/main/%EB%B0%91%EB%B0%94%EB%8B%A5%EB%94%A5%EB%9F%AC%EB%8B%9D_4%EC%9E%A5_%EC%8B%A0%EA%B2%BD%EB%A7%9D_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#2025-05-16
#여기서 학습이란 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획듯하는 것을 뜻함.
#이번 장에서는 신경망이 학습할 수 있도록 해주는 지표인 손실 함수를 소개함.
#이 손실 함수의 결괏값을 가장 작게 만드는 가중치 매개변수를 찾는게 이번 챕터의 목표
import numpy as np
import importlib
importlib.invalidate_caches()


In [ ]:
#손실함수
#오차 제곱합 == E = 1/2 *np.sum((y - t)**2)

In [ ]:
#예시

y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] #예측값
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] #정답값
#이 배열의 원소는 첫 번째 인덱스부터 순서대로 숫자 0 1 2 일 때의 값
#여기에서 ㅣㄴ경망의 출력 y는 softmax함수의 출력 고로 확률값
#이 예시에서 이미지가 0일확률은 0.1 1일 확률은 0.05 2일 확률은 0.6
#정답 레이블인 t는 정답을 가리키는 위치의 원소는 1로 그 외에는 0으로 ㅍ기함 여기에서는 숫자 2에 해당되는 원소의 값이
#1이므로 정답이 2임을 알수있다./

In [ ]:
#손실함수
#오차제곱합
def sum_squares_error(y,t):
  return 0.5 * np.sum((y-t)**2)

y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] #예측값
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] #정답값

#예 : 2일 확률이 가장 높다고 추정함 (0.6)
print(sum_squares_error(np.array(y),np.array(t))) # 0.0975 오차값이 낮게 나온 모습

#예2 : 7일 확률이 가장 높다고 추정함 (0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(sum_squares_error(np.array(y),np.array(t))) # 0.5975 오차가 높게 나온 모습.

0.09750000000000003
0.5975


In [ ]:
#교차 엔트로피 오차 cross entropy error CEE
#여기서 log는 자연로그 y는 신경망의 출력, t는 정답 레이블
#또 t는 정답에 해당하는 인덱스의 원소만 1이고 다른 건 0임. 그래서 실적으로 정답일 때 (t가 1일때의 y)의 자연로그를 계산함
#만약 y랑 t둘다 1이면 0을 출력함 오차가 0이라는 뜻.
#E = -sum.np(t * np.log(y))

def cross_entropy_error(y,t):
  y = np.array(y)
  t = np.array(t)
  delta = 1e-7
  return -np.sum(t * np.log(y+delta))
#아주 작은 값인 delta를 더한 이유는 np.log 함수에 0을 입력하면 -inf가 출력되어
#이를 방지 하기 위해

y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] #예측값
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] #정답값

print(cross_entropy_error(y,t)) # 0.51

y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

print(cross_entropy_error(y,t)) # 2.30
#밑의 오차가 2.3으로 훨씬 크게 나옴 앞서 오차제곱합의 판단과 일치함.

0.510825457099338
2.302584092994546


In [ ]:
#2025-05-20 외박 및 당직이슈.
#미니배치 학습
#지금까지 데이터 하나의 대한 손실함수만 생각해왔으니, 이제 훈련 데이터 모두에 대한 손실함수의 합을 구해보자
import numpy as np
import importlib
import sys
import pickle
importlib.invalidate_caches()
sys.path.append('/content/drive/MyDrive/dataset')

from mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize= True, one_hot_label= True) #정규화 on 원하라벨 = 정답 위치 원소만1

print(x_train.shape) # 784개의 픽셀의 가진 이미지 6만개 학습용 데이터
print(t_train.shape) # 그에 대응 되는 정답 데이터 6만개.
print(x_test.shape) #여긴 테스트 데이터 1만개
print(t_test.shape)

(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)


In [ ]:
#위의 많은 데이터중 10장만 빼내는법
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) # 0~6만 수중에 무작위 10개 뽑
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [ ]:
#배치용 교차 엔트로피 오차 구현하기

def cross_entropy_error(y,t):
  if y.ndim == 1: #1차원이라면.
    t = t.reshape(1,t.size)
    y = y.reshape(1,y.size) # y가 1차원이라면, 즉 데이터 하나에 대한 오차를 구하는경우 데이터의 형상을 바꿈
    #(3,)같은 형태를 (1,3)으로 바꾼다.

  batch_size = y.shape[0]
  return -np.sum(t * np.log(y + 1e-7)) / batch_size


In [ ]:
#2025-05-21
#수치미분

#나쁜 구현의 예
def numerical_diff(f,x):
  h = 10e-50
  return (f(x+h) - f(x) / h) #10e-50을 컴퓨터에서는 표현을 못함. 그래서 미분구현이 살짝 다름

#옳은 구현
def numerical_diff(f,x):
  h = 1e-4
  return (f(x+h) - f(x-h)) / (2*h)

In [ ]:
#y = 0.01x^2 + 0.1x 미분
def function_1(x):
  return 0.01*x**2 + 0.1*x

numerical_diff(function_1,5) #0.19999999999990898

numerical_diff(function_1, 10) #0.2999999999986347


0.2999999999986347

In [ ]:
#편미분은 어떻게?
#x0에 대한 편미분을 구하고 싶으면 x만 변수로 놔두고 x1은 상수로 바꿔버리면 된다.

#x0 = 3 x1 = 4일때 x0에 대한 편미분
def function_tmp1(x0):
  return x0*x0 + 4.0**2.0

numerical_diff(function_tmp1,3.0) #6.00000000000378

#x0 = 3 x1 = 4일때 x1에 대한 편미분
def function_tmp2(x1):
  return 3.0**2 + x1**2.0

numerical_diff(function_tmp1,4.0) #7.999999999999119


7.999999999999119

In [ ]:
#위에서 x0과 x1의 편미분을 변수별로 따로 계산했는데 x0과 x1의 편미분을 동시에 계산할려면?
#기울기 벡터 구하는법

def numerical_gradient(f,x):
  h = 1e-4
  grad = np.zeros_like(x)

  for idx in range(x.size):
    tmp_val = x[idx]
    #f(x+h)계산
    x[idx] = tmp_val + h
    fxh1 = f(x)
#2025-05-24
    x[idx] = tmp_val - h
    fxh2 = f(x)
    grad[idx] = (fxh1 - fxh2) / (2*h)
    #아까 위에서 구현한 수치미분이랑 동일함.

  return grad

def func_ex2(x):
  return x[0] ** 2 + x[1]**2

numerical_gradient(func_ex2,np.array([3.0,4.0])) # 정상적으로 6과 8을 출력하는 모습

array([6., 8.])

In [ ]:
#경사하강법 gradient descent
#기계학습 문제 대부분은 학습 단계에서 최적의 매개변수를 찾아내는데 신경망도 똑같다. 여기서 최적이란
#손실함수가 최솟값이 될 때의 매개변수의 값인데 이걸 찾아내는게 경사하강법이다.
# 경사하강법의 식 x[0] = x[0] - α(∂f/∂x[0]) , x[1] = x[1] - α(∂f/∂x[1])
#α = 하이퍼 파라미터인 러닝레이트 학습 속도를 조절해준다.

def gradient_descent(f,init_x,lr = 0.01,step_num = 100): #d init_x = 초깃값,러닝레이트 0.01 반복횟수는 100번
  x = init_x

  for i in range(step_num):
    grad = numerical_gradient(f,x)
    x -= lr*grad

  return x

#문제 : 경사하강법으로f(x[0],x[1]) = x[0]^2 + x[1]^2의 최솟값을 구하라.
#일단 답은 당연히 [0,0]임

def function_2(x):
  return x[0] ** 2 + x[1] **2

gradient_descent(function_2, init_x= np.array([-3.0,4.0]),lr = 0.1,step_num=100) # [-0.0005, -0.0005]

array([-0.0005, -0.0005])

In [ ]:
#신경망에서의 기울기
#신경망 학습에서도 기울기를 구해야 하는데, 여기서 말하는 기울기는 가중치 매개변수에 대한 손실 함수의 기울기이다
#걍 간단하게 말하면 손실함수를 미분하여 w파라미터를 업데이트한다는 소리
#예를 들어 형상이 2x3, 가중치가 W(벡터), 손실 함수가 L인 신경망을 예시로 들어보면
#∂L/∂W (각 원소에 대한 편미분 예를들어 1행 1번째 원소인 (∂L/∂w11)는 w11를 조금 변경했을때 손실함수 L이 얼마나 변하나임
# W = np.array([[w11,w12,w13],[w21,w22,w23]])

#간단한 실제 기울기 구하는 예제

from func import softmax, cross_entropy_error, sigmoid, numerical_gradient

class simpleNet:
  def __init__(self):
    self.W = np.random.randn(2,3) #정규분포로 초기화

  def predict(self,x):
    return np.dot(x,self.W)

  def loss(self, x,t):
    z = self.predict(x)
    y = softmax(z)
    loss = cross_entropy_error(y,t)

    return loss

net = simpleNet()
print(net.W)

x = np.array([0.6,0.9])
p = net.predict(x)
print(p)

np.argmax(p)

t = np.array([0,0,1]) #원 핫 코딩
print(net.loss(x,t))

def f(W):
  return net.loss(x,t)





[[ 0.64684821 -1.71638144 -0.28383369]
 [ 0.73393708  2.06827318  0.67187364]]
[1.04865229 0.831617   0.43438606]
1.4669516813031438


In [ ]:
#2025-05-25
#2층 신경망 클래스 구현
class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):#입력층,은닉층,출력층의뉴련수
    #가중치 초기화
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size)
    self.params['b1'] = np.zeros(hidden_size)

    self.params['W2'] = weight_init_std * np.random.randn(hidden_size,output_size)
    self.params['b2'] = np.zeros(output_size)

  def perdict(self,x):
    W1,W2 = self.params['W1'], self.params['W2']
    b1,b2 = self.params['b1'], self.params['b2']

    a1 = np.dot(x,W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1,W2) + b2
    y = softmax(a2)

    return y

  def loss(self,x,t):
    y = self.perdict(x)

    return cross_entropy_error(y,t)

  def accuracy(self,x,t):
    y = self.predict(x)
    y = np.argmax(y, axis = 1)
    t = np.argmax(t, axis = 1)

    accuracy = np.sum(y==t) / float(x.shape[0])

    return accuracy
#x = 입력 t = 정답 레이블
  def numerical_gradient(self,x,t):
    loss_W = lambda W: self.loss(x,t)
    grads = {}
    # 기울기 벡터 구하는 부분
    grads['W1'] = numerical_gradient(loss_W,self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W,self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W,self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W,self.params['b2'])
    #numerical_gradient함수의 인자로 loss함수와 W1값을 줘서 W1의 원소값을 h만큼 늘리면 얼마나 오차가 생기는지로
    #기울기를 측정함 이를 기반으로 기울기 벡터를 구함.

    return grads

In [ ]:
net = TwoLayerNet(input_size=784,hidden_size=100,output_size=10)

x = np.random.rand(100,784) #더미 데이터 100장
t = np.random.rand(100,10) #정답 데이터 100장

y = net.perdict(x)

grads = net.numerical_gradient(x,t) #약 3분이나 걸린다.
#역전파를 배우면 고속으로 기울기 벡터를 찾는 함수를 만들것이다.